In [3]:
from z3 import Solver, Bool, Xor, If, Or, sat
import numpy as np
from functools import reduce

# Stabilizer Lists => Coordinates

In [30]:
def get_stab_coords(grid):
    stabilizers = {}
    num_cols = len(grid[0])  # Number of columns in the grid

    # Populate the initial stabilizers dictionary
    for i, row in enumerate(grid):
        for j, letter in enumerate(row):
            if letter != '.':
                if letter not in stabilizers:
                    stabilizers[letter] = [[]]  # Initialize with a list containing an empty list
                stabilizers[letter][0].append(i * num_cols + j)  # Append to the first list

    # Iterate through each letter and perform the transformation
    for letter, lists in stabilizers.items():
        original_list = lists[0]

        # Precompute row and column indices for the original list
        indices = [(value // num_cols, value % num_cols) for value in original_list]

        # Generate all transformed lists in one go
        transformed_lists = [
            [
                i * num_cols + ((j + n) % num_cols)
                for i, j in indices
            ]
            for n in range(1, num_cols)
        ]

        # Extend the lists with all the new transformed lists
        lists.extend(transformed_lists)

    return stabilizers

def generate_stabilizer_matrix(grid):
    # Step 1: Prepare stabilizer data
    stab_coords = get_stab_coords(grid)
    num_positions = len(grid) * len(grid[0])
    stabilizers = list(stab_coords.keys())

    matrix = []
    # Step 2: Fill the matrix
    for stabilizer in stabilizers:
        for coord_list in stab_coords[stabilizer]:
            row = [0] * num_positions
            for position in coord_list:
                row[position] = 1
            matrix.append(row)

    return matrix

In [31]:
def generate_translations(base_stabilizers, num_rows, num_cols):
    stabilizers = {}
    # base_stabilizers is a dict: letter -> list of (row, col) tuples
    
    # Convert each stabilizer's base coordinate list into full sets of translated coordinates
    for letter, coords in base_stabilizers.items():
        # Each entry in stabilizers[letter] will be a list of positions in linear index form
        # Start with the original position list
        original_list = [r * num_cols + c for (r, c) in coords]
        stabilizers[letter] = [original_list]

        # Generate translations for each shift n from 1 to num_cols-1
        transformed_lists = [
            [
                i * num_cols + ((j + n) % num_cols)
                for (i, j) in coords
            ]
            for n in range(1, num_cols)
        ]
        
        # Add these transformed lists
        stabilizers[letter].extend(transformed_lists)

    return stabilizers


def generate_stabilizer_matrix_from_dict(base_stabilizers, num_rows, num_cols):
    stab_coords = generate_translations(base_stabilizers, num_rows, num_cols)
    num_positions = num_rows * num_cols
    stabilizers = list(stab_coords.keys())

    matrix = []
    for stabilizer in stabilizers:
        for coord_list in stab_coords[stabilizer]:
            row = [0] * num_positions
            for position in coord_list:
                row[position] = 1
            matrix.append(row)

    return matrix


def gf2_add_row(r1, r2):
    return [a ^ b for a, b in zip(r1, r2)]

def gf2_rref(matrix):
    M = [row[:] for row in matrix]
    rows = len(M)
    cols = len(M[0]) if rows > 0 else 0

    pivot_positions = []
    pivot_row = 0
    for col in range(cols):
        if pivot_row >= rows:
            break
        # Find pivot
        pivot = -1
        for r in range(pivot_row, rows):
            if M[r][col] == 1:
                pivot = r
                break
        if pivot == -1:
            continue

        # Swap to put pivot in place
        if pivot != pivot_row:
            M[pivot_row], M[pivot] = M[pivot], M[pivot_row]

        # Eliminate down the column
        for r in range(rows):
            if r != pivot_row and M[r][col] == 1:
                M[r] = gf2_add_row(M[r], M[pivot_row])

        pivot_positions.append(col)
        pivot_row += 1
    return M, pivot_positions

def convert_stabilizer_matrix_to_generator_matrix(H):
    R, pivots = gf2_rref(H)
    n = len(H[0])
    r = len(pivots)
    k = n - r
    if k <= 0:
        return []

    pivot_set = set(pivots)
    free_cols = [c for c in range(n) if c not in pivot_set]

    G = []
    for free_col in free_cols:
        vec = [0]*n
        vec[free_col] = 1
        # Solve pivot vars
        for row_idx, pcol in enumerate(pivots):
            rhs = 0
            for c, val in enumerate(R[row_idx]):
                if c != pcol and val == 1:
                    rhs ^= vec[c]
            vec[pcol] = rhs
        G.append(vec)

    return G


In [63]:
grid_22 = [
    ['.', '.', '.', '.', '.', 'F', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', 'E', '.', '.', 'F', '.', 'F', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
    ['E', '.', 'E', '.', '.', '.', 'F', '.', '.', '.', '.', '.', '.', 'D', '.', '.', '.'],
    ['E', '.', '.', '.', '.', '.', '.', '.', '.', 'C', '.', '.', '.', '.', 'D', '.', '.'],
    ['.', '.', '.', '.', '.', 'B', '.', '.', '.', '.', 'C', '.', 'D', '.', 'D', '.', '.'],
    ['.', 'A', '.', '.', '.', '.', '.', '.', 'C', '.', 'C', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', 'B', 'B', 'B', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
    ['A', 'A', 'A', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
]

base_22 = {
    'F': [(0, 5), (1, 4), (1, 6), (2, 6)],
    'E': [(1, 1), (2, 0), (2, 2), (3, 0)],
    'D': [(2, 13), (3, 14), (4, 12), (4, 14)],
    'C': [(3, 9), (4, 10), (5, 8), (5, 9)],
    'B': [(4, 5), (6, 4), (6, 5), (6, 6)],
    'A': [(5, 1), (7, 0), (7, 1), (7, 2)]
}


grid_16 = [
    # 0    1.   2.   3.   4.   5.   6.   7.   8.   9.  10.   11.  12.  13.  14.  15.  16
    ['.', 'E', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
    ['E', 'E', 'E', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', 'D', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.', '.', 'C', '.', '.', 'D', '.', 'D', '.', '.'],
    ['.', '.', '.', '.', '.', 'B', '.', '.', 'C', '.', 'C', '.', '.', 'D', '.', '.', '.'],
    ['.', 'A', '.', '.', 'B', '.', '.', '.', 'C', '.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', 'A', '.', '.', 'B', '.', 'B', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
    ['A', '.', 'A', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
]

base_16 = {
    'E': [(0, 1), (1, 0), (1, 1), (1, 2)],
    'D': [(1, 13), (2, 12), (2, 14), (3, 13)],
    'C': [(2, 9), (3, 8), (3, 10), (4, 8)],
    'B': [(3, 5), (4, 4), (5, 4), (5, 6)],
    'A': [(4, 1), (5, 1), (6, 0), (6, 2)]
}


grid_12 = [
    ['.', 'D', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
    ['D', 'D', 'D', '.', '.', '.', '.', '.', '.', 'C', '.', '.', '.'],
    ['.', '.', '.', '.', '.', 'B', '.', '.', 'C', '.', 'C', '.', '.'],
    ['.', 'A', '.', '.', 'B', '.', '.', '.', '.', '.', 'C', '.', '.'],
    ['A', '.', '.', '.', 'B', '.', 'B', '.', '.', '.', '.', '.', '.'],
    ['A', '.', 'A', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
]

base_12 = {
    'D': [(0, 1), (1, 0), (1, 1), (1, 2)],
    'C': [(1, 9), (2, 8), (2, 10), (3, 10)],
    'B': [(2, 5), (3, 4), (4, 4), (4, 6)],
    'A': [(3, 1), (4, 0), (5, 0), (5, 2)]
}

grid_9 = [
    ['.', '.', '.', '.', '.', '.', '.', '.', '.', 'C', '.'],
    ['.', '.', '.', '.', '.', 'B', '.', '.', 'C', '.', 'C'],
    ['.', 'A', '.', '.', '.', '.', 'B', '.', 'C', '.', '.'],
    ['.', '.', 'A', '.', 'B', '.', 'B', '.', '.', '.', '.'],
    ['A', '.', 'A', '.', '.', '.', '.', '.', '.', '.', '.']
]

base_9 = {
    'C': [(0, 9), (1, 8), (1, 10), (2, 8)],
    'B': [(1, 5), (2, 6), (3, 4), (3, 6)],
    'A': [(2, 1), (3, 2), (4, 0), (4, 2)]
}

grid_5 = [
    ['.', '.', '.', 'B', '.'],
    ['.', 'A', 'B', 'B', '.'],
    ['A', '.', '.', '.', 'B'],
    ['A', 'A', '.', '.', '.']
]

base_5 = {
    'B': [(0, 3), (1, 2), (1, 3), (2, 4)],
    'A': [(1, 1), (2, 0), (3, 0), (3, 1)]
}

In [24]:
import logging
from functools import reduce
from z3 import Optimize, Bool, Xor, Sum, If, Or, is_true

# Configure logging to output to console and file
logging.basicConfig(
    level=logging.INFO,  # Set to INFO to capture all logging messages
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('hamming_distance.log'),
        logging.StreamHandler()  # Also output to console
    ]
)

def compute_min_hamming_distance(G):
    # Initialize Z3 optimizer
    optimizer = Optimize()

    # Number of rows in generator matrix G (number of codewords)
    k = len(G)
    n = len(G[0])  # Length of each codeword

    print(f"Generator matrix has {k} rows and {n} columns.")

    # Define Boolean variables for each row of G
    b = [Bool(f'b_{i}') for i in range(k)]

    # Add constraint: At least one row is selected (non-zero codeword)
    optimizer.add(Or(b))

    # Helper function for XOR over multiple terms
    def safe_xor(terms):
        if len(terms) == 0:
            return False  # Represents zero in Boolean logic
        elif len(terms) == 1:
            return terms[0]
        else:
            return reduce(lambda x, y: Xor(x, y), terms)

    # Precompute codeword structure
    codeword = []
    for j in range(n):
        terms = [b[i] for i in range(k) if G[i][j] == 1]
        xor_result = safe_xor(terms)
        codeword.append(xor_result)
        logging.debug(f"Computed codeword bit {j}: {xor_result}")

    # Define Hamming weight
    hamming_weight = Sum([If(bit, 1, 0) for bit in codeword])

    # Ensure non-zero Hamming weight (exclude zero codeword)
    optimizer.add(hamming_weight > 0)

    # Set the objective to minimize the Hamming weight
    optimizer.minimize(hamming_weight)

    result = optimizer.check()

    if result == sat:
        model = optimizer.model()
        min_distance = model.evaluate(hamming_weight).as_long()
        selected_rows = [i for i in range(k) if is_true(model.evaluate(b[i]))]
        print(f"Minimum non-zero Hamming distance found: {min_distance}")
        print(f"Selected rows in G contributing to the codeword: {selected_rows}")

        # Compute the codeword values
        computed_codeword = [is_true(model.evaluate(bit)) for bit in codeword]
        binary_codeword = [1 if bit else 0 for bit in computed_codeword]

        # Log the contributing rows
        print("\nContributing rows from G:")
        for idx in selected_rows:
            print(f"Row {idx}: {G[idx]}")

        # Log the resulting codeword
        print("\nXOR Result (codeword):")
        print(f"{binary_codeword}")

        return min_distance
    else:
        print("No valid codeword found.")
        return "No valid codeword found"


In [54]:
H = generate_stabilizer_matrix_from_dict(base_5, 4, 5)
G = convert_stabilizer_matrix_to_generator_matrix(H)
min_distance = compute_min_hamming_distance(G)
print("Minimum distance:", min_distance)

Generator matrix has 10 rows and 20 columns.
Minimum non-zero Hamming distance found: 5
Selected rows in G contributing to the codeword: [6]

Contributing rows from G:
Row 6: [0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

XOR Result (codeword):
[0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Minimum distance: 5


In [53]:
H = generate_stabilizer_matrix_from_dict(base_9, 5, 11)
G = convert_stabilizer_matrix_to_generator_matrix(H)
min_distance = compute_min_hamming_distance(G)
print("Minimum distance:", min_distance)

Generator matrix has 22 rows and 55 columns.
Minimum non-zero Hamming distance found: 9
Selected rows in G contributing to the codeword: [9, 18]

Contributing rows from G:
Row 9: [1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Row 18: [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

XOR Result (codeword):
[1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Minimum distance: 9


In [57]:
H = generate_stabilizer_matrix_from_dict(base_12, 6, 13)
G = convert_stabilizer_matrix_to_generator_matrix(H)
min_distance = compute_min_hamming_distance(G)
print("Minimum distance:", min_distance)

Generator matrix has 26 rows and 78 columns.
Minimum non-zero Hamming distance found: 12
Selected rows in G contributing to the codeword: [8, 21]

Contributing rows from G:
Row 8: [0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Row 21: [0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

XOR Result (codeword):
[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
Minimum distance: 12


In [64]:
H = generate_stabilizer_matrix_from_dict(base_16, 7, 17)
G = convert_stabilizer_matrix_to_generator_matrix(H)
min_distance = compute_min_hamming_distance(G)
print("Minimum distance:", min_distance)

Generator matrix has 34 rows and 119 columns.
Minimum non-zero Hamming distance found: 16
Selected rows in G contributing to the codeword: [14]

Contributing rows from G:
Row 14: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

XOR Result (codeword):
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Minimum distance: 16


In [19]:
H = generate_stabilizer_matrix_from_dict(base_22, 8, 17)
G = convert_stabilizer_matrix_to_generator_matrix(H)
min_distance = compute_min_hamming_distance(G)
print("Minimum distance:", min_distance)

Generator matrix has 34 rows and 136 columns.
No valid codeword found.
Minimum distance: No valid codeword found
